# General Preprocessing 
## 해당 내용은 정략적으로 데이터를 증가시킬 때 (실시간으로 안할때)
## 특히, darknet platform에서 detection할 때

In [1]:
import cv2
import numpy as np
import pandas as pd
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
from PIL import Image
import os
import glob
import shutil
import PIL.ImageOps 

# Augmentation - shift, rotation, inversion, and etc (필요한 함수)

In [2]:
import os
import cv2
from PIL import Image, ImageEnhance
import os
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from scipy.ndimage import filters,interpolation
from scipy import misc
import shutil
import imutils
from scipy.ndimage import zoom
from IPython.display import clear_output

_max_filiter_size1 = 7  # for avg_blur and gaussain_blur
_max_filiter_size2 = 5  # for avg_blur and gaussain_blur
_max_filiter_size3 = 3  # for avg_blur and gaussain_blur
_sigma1 = 0  # for gaussain_blur
_sigma2 = 0  # for gaussain_blur
_sigma3 = 0  # for gaussain_blur

_mean = 0  # for gaussain_noise
_var = 0.1  # for gaussain_noise

_x_min_shift_piexl = -20  # for img_shift
_x_max_shift_piexl = 20  # for img_shift
_y_min_shift_piexl = -20  # for img_shift
_y_max_shift_piexl = 20  # for img_shift
_fill_pixel = 0  # for img_shift and img_rotation: black

_min_angel = -20  # for img_rotation
_max_angel = 20  # for img_rotation
_min_scale = 0.9  # for img_rotation
_max_scale = 1.1  # for img_rotation

_min_s = -10  # for img_contrast
_max_s = 10  # for img_contrast
_min_v = -10  # for img_contrast
_max_v = 10  # for img_contrast

_min_h = -30  # for img_color
_max_h = 30  # for img_color

_generate_quantity = 10

def gaussain_blur(img, fitersize, sig):
    size = (fitersize,fitersize)    
    return cv2.GaussianBlur(img, size, sig)

def img_shift(image,x0,x1,y0,y1,mode='nearest',cval=0):
    cols = image.shape[0]
    rows = image.shape[0]
    M = np.float32([[1,0,x1],[0,1,y1]])
    dst = cv2.warpAffine(image,M,(cols,rows))
    return dst

def img_inv(image):
    inverted_image = abs(255 - image)
    return inverted_image

def noise_generator (noise_type,image):
    """
    Generate noise to a given Image based on required noise type
    
    Input parameters:
        image: ndarray (input image data. It will be converted to float)
        
        noise_type: string
            'gauss'        Gaussian-distrituion based noise
            'poission'     Poission-distribution based noise
            's&p'          Salt and Pepper noise, 0 or 1
            'speckle'      Multiplicative noise using out = image + n*image
                           where n is uniform noise with specified mean & variance
    """
    row,col,ch= image.shape
    if noise_type == "gauss":       
        mean = 0.0
        var = 0.01
        sigma = var**0.5
        gauss = np.array(image.shape)
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy.astype('uint8')
    elif noise_type == "s&p":
        s_vs_p = 0.5
        amount = 0.004
        out = image
        # Generate Salt '1' noise
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 255
        # Generate Pepper '0' noise
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_type == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_type =="speckle":
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy
    else:
        return image

def clipped_zoom(img, zoom_factor, **kwargs):
    #print(img.shape)
    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out 

# Image & Mask image - geometric pre-processing

In [22]:
zoom_factor = 1.2
     
def process_mask():
        pathimage =     #orinal image path folder name (MASK 폴더)
        savepathimage = #pre-processing svae path folder name

        data_dir = path 
        img_lst = os.listdir(data_dir)
        for k,name in enumerate(img_lst):
            abs_path = os.path.join(data_dir, name)     

            clear_output(wait=False)
            print (k+1," / ",len(os.listdir(path)), " : ",name)

            if name.endswith("jpg"):
                img = cv2.imread(abs_path)              
                prefix, suffix = abs_path.split('.')
                filename = name.split(".")[0].split("_")

                #10단계
                for k in range(10):
                    _x_max_shift_piexl = 20 * k
                    _y_max_shift_piexl = 20 * k
                    _x_min_shift_piexl = -20 * k
                    _y_min_shift_piexl = -20 * k
                    _max_angel = 1 * k 
                    _min_angel = -1 * k

                    cv2.imwrite('%s%s.%s' % (savepath1+"/"+filename[0]+"-"+filename[1],'_shift'+str(k)+"_"+filename[2], suffix),
                                img_shift(img, 0, _x_max_shift_piexl, 0, _y_max_shift_piexl,_fill_pixel))
                    cv2.imwrite('%s%s.%s' % (savepath1+"/"+filename[0]+"-"+filename[1],'_shift1'+str(k)+"_"+filename[2], suffix),
                                img_shift(img, 0, _x_min_shift_piexl, 0, _y_min_shift_piexl,_fill_pixel))              
                    cv2.imwrite('%s%s.%s' % (savepath1+"/"+filename[0]+"-"+filename[1],'_rotation'+str(k)+"_"+filename[2], suffix),
                                imutils.rotate(img, _max_angel))
                    cv2.imwrite('%s%s.%s' % (savepath1+"/"+filename[0]+"-"+filename[1],'_rotation1'+str(k)+"_"+filename[2], suffix),
                                imutils.rotate(img, _min_angel))
                imagecov = np.array(img)
                cv2.imwrite('%s%s.%s' % (savepath1+"/"+filename[0]+"-"+filename[1], '_zoom'+"_"+filename[2], suffix),
                            clipped_zoom(imagecov, zoom_factor))


def process_image():
  
        pathimage =     #orinal image path folder name
        savepathimage = #pre-processing svae path folder name
        
        data_dir = pathimage 
        img_lst = os.listdir(data_dir)
        for k, name in enumerate(img_lst):
            abs_path = os.path.join(data_dir, name)   
            clear_output(wait=False)
            print (k+1," / ",len(os.listdir(pathimage)), " : ",name)

            if name.endswith("png"):
                img = cv2.imread(abs_path)              
                prefix, suffix = abs_path.split('.')
                filename = name.split(".")[0].split("_")
                for k in range(10):
                    _x_max_shift_piexl = 20 * k
                    _y_max_shift_piexl = 20 * k
                    _x_min_shift_piexl = -20 * k
                    _y_min_shift_piexl = -20 * k
                    _max_angel = 1 * k 
                    _min_angel = -1 * k

                    cv2.imwrite('%s%s.%s' % (savepathimage+"/"+filename[0],'_shift'+str(k), suffix),
                                img_shift(img, 0, _x_max_shift_piexl, 0, _y_max_shift_piexl,_fill_pixel))
                    cv2.imwrite('%s%s.%s' % (savepathimage+"/"+filename[0],'_shift1'+str(k), suffix),
                                img_shift(img, 0, _x_min_shift_piexl, 0, _y_min_shift_piexl,_fill_pixel)) 
                    cv2.imwrite('%s%s.%s' % (savepathimage+"/"+filename[0],'_rotation'+str(k), suffix),
                                imutils.rotate(img, _max_angel))
                    cv2.imwrite('%s%s.%s' % (savepathimage+"/"+filename[0],'_rotation1'+str(k), suffix),
                                imutils.rotate(img, _min_angel))
                    imagecov = np.array(img)
                    cv2.imwrite('%s%s.%s' % (savepathimage+"/"+filename[0], '_zoom', suffix),
                                clipped_zoom(imagecov, zoom_factor))

In [23]:
#process_mask() #detection or segmentation이 필요할 때 사용
process_image() #

581  /  637  :  o2532483474391730372537_01Nodule.jpg


# Image & Mask image - photographic pre-processing
## Sharp & Blur & noise & brightness & contrast & inversion

In [24]:
def process_aug():  

        pathimage = #raw image folder path
        savepathimage = #save image folder path
            
        img_lst = os.listdir(pathimage)
        for k, name in enumerate(img_lst):
            abs_path = os.path.join(pathimage, name)   
            clear_output(wait=False)
            print (k+1," / ",len(os.listdir(pathimage)), " : ",name)
            
            if name.endswith(".jpg"):
                img = cv2.imread(abs_path)
                aff = name.split("_")

                prefix, suffix = abs_path.split('.')
                filename = name.split(".")
                cv2.imwrite('%s_%s.%s' % (savepathimage+"/"+filename[0], 'inv', suffix),img_inv(img))         
                cv2.imwrite('%s_%s.%s' % (savepathimage+"/"+filename[0], 'blur1', suffix), gaussain_blur(img, _max_filiter_size1, _sigma1))  
                cv2.imwrite('%s_%s.%s' % (savepathimage+"/"+filename[0], 'blur2', suffix), gaussain_blur(img, _max_filiter_size2, _sigma2))  
                cv2.imwrite('%s_%s.%s' % (savepathimage+"/"+filename[0], 'blur3', suffix), gaussain_blur(img, _max_filiter_size3, _sigma3))
                cv2.imwrite('%s_%s.%s' % (savepathimage+"/"+filename[0], 'noise1', suffix), noise_generator('gauss', img))

                img02 = Image.open(abs_path)
                ImageEnhance.Sharpness(img02).enhance(2.0).save('%s_%s.%s' % (savepathimage+"/"+filename[0], 'sharpness1', suffix))
                ImageEnhance.Sharpness(img02).enhance(2.5).save('%s_%s.%s' % (savepathimage+"/"+filename[0], 'sharpness2', suffix))
                ImageEnhance.Sharpness(img02).enhance(3.0).save('%s_%s.%s' % (savepathimage+"/"+filename[0], 'sharpness3', suffix))   
                ImageEnhance.Brightness(img02).enhance(1.5).save('%s_%s.%s' % (savepathimage+"/"+filename[0], 'brightness2', suffix))
                ImageEnhance.Contrast(img02).enhance(1.5).save('%s_%s.%s' % (savepathimage+"/"+filename[0], 'contrast2', suffix))

In [25]:
process_aug()

686  /  813  :  o2532483474391730372537_01Nodule.jpg
